# Materializers

The precise way that data passes between the steps is dictated by `materializers`. The data that flows through steps 
are stored as artifacts and artifacts are stored in artifact stores. The logic that governs the reading and writing of 
data to and from the artifact stores lives in the materializers. In order to control more precisely how data 
flows between steps, one can simply create a custom materializer by sub-classing the `BaseMaterializer` class.

Materializers are also necessary when you want Custom objects to be passed between steps. Within this tutorial you'll learn how to build your own materializers.

In [1]:
%%capture
!pip install zenml

In [2]:
import os
from typing import Type

from zenml.artifacts import DataArtifact
from zenml.pipelines import pipeline
from zenml.steps import step

We'll start off with a custom class called `MyObj`. You can imagine this as a complex class that describes your training data with all its relevant metadata.

In [3]:
class MyObj:
    """Your custom implementation of a data object"""
    def __init__(self, name: str):
        self.name = name

Next we will construct a basic pipeline in which the an instance of this `MyObj` class is passed between the pipeline steps

In [4]:
@step
def step1() -> MyObj:
    """Step that returns one of your data objects"""
    return MyObj("aria_the_cat")

@step
def step2(my_obj: MyObj):
    """Step that uses one of your data objects as input"""
    print(my_obj.name)
    
@pipeline
def pipe(step1, step2):
    """Connecting the steps together in a pipeline"""
    custom_data_object = step1()
    step2(custom_data_object)

Lets run this pipeline and see how it behaves when it does not know how to write and read the `MyObj`instance to and from the Artifact Store.

In [5]:
# This will fail - don't worry, we will fix this together
pipe(
    step1=step1(),
    step2=step2()
).run()

Creating run for pipeline: `pipe`
Cache enabled for pipeline `pipe`
Using stack `secrets_stack2` to run pipeline `pipe`...


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_340086/1509130869.py:2 in <cell line: 2>                                   │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_340086/1509130869.py'                │
│ /home/apenner/PycharmProjects/zenml/src/zenml/pipelines/base_pipeline.py:378 in run       │
│                                                                                           │
│   375 │   │   self._reset_step_flags()                                                    │
│   376 │   │   self.validate_stack(stack)                                                  │
│   377 │   │                                                                               │
│ ❱ 378 │   │   return stack.deploy_pipeline(                                               │
│   379 │   │   │   self, runtime_configuration=runtime_configuration                       │
│   380 │   │   )                                                                           │
│   381                                                                                     │
│                                                                                           │
│ /home/apenner/PycharmProjects/zenml/src/zenml/stack/stack.py:361 in deploy_pipeline       │
│                                                                                           │
│   358 │   │   )                                                                           │
│   359 │   │   start_time = time.time()                                                    │
│   360 │   │                                                                               │
│ ❱ 361 │   │   return_value = self.orchestrator.run_pipeline(                              │
│   362 │   │   │   pipeline, stack=self, runtime_configuration=runtime_configuration       │
│   363 │   │   )                                                                           │
│   364                                                                                     │
│                                                                                           │
│ /home/apenner/PycharmProjects/zenml/src/zenml/orchestrators/local/local_orchestrator.py:8 │
│ 3 in run_pipeline                                                                         │
│                                                                                           │
│    80 │   ) -> Any:                                                                       │
│    81 │   │   """Runs a pipeline locally"""                                               │
│    82 │   │                                                                               │
│ ❱  83 │   │   tfx_pipeline: TfxPipeline = create_tfx_pipeline(pipeline, stack=stack)      │
│    84 │   │                                                                               │
│    85 │   │   if runtime_configuration is None:                                           │
│    86 │   │   │   runtime_configuration = RuntimeConfiguration()                          │
│                                                                                           │
│ /home/apenner/PycharmProjects/zenml/src/zenml/orchestrators/utils.py:45 in                │
│ create_tfx_pipeline                                                                       │
│                                                                                           │
│    42 ) -> tfx_pipeline.Pipeline:                                                         │
│    43 │   """Creates a tfx pipeline from a ZenML pipeline."""                             │
│    44 │   # Connect the inputs/outputs of all steps in the pipeline                       │
│ ❱  45 │   zenml_pipeline.connect(**zenml_pipeline.

As you can see, you are getting a **StepInterfaceError**. Step1 is trying to write its output to the Artifact Store but can't find a Materializer to do so. Let's write one!


For Custom Materializers allyou need to do is inherit from the ZenML BaseMaterializer, define the ASSOCIATED_TYPES and implement the two methods (handle_input, handle_return). Make sure to use the zenml fileio for the implementation of the read/write. This ensures that this will work, even when you switch out the Artifact Store at a later point in time.

In [6]:
from zenml.materializers.base_materializer import BaseMaterializer
from zenml.io import fileio



class MyMaterializer(BaseMaterializer):
    """Materializer for you data object"""
    # Within the associated types you specify which object this materializer
    # is built to handle
    ASSOCIATED_TYPES = (MyObj,)
    ASSOCIATED_ARTIFACT_TYPES = (DataArtifact,)

    def handle_input(self, data_type: Type[MyObj]) -> MyObj:
        """Read from artifact store"""
        super().handle_input(data_type)
        with fileio.open(os.path.join(self.artifact.uri, "data.txt"), "r") as f:
            name = f.read()
        return MyObj(name=name)

    def handle_return(self, my_obj: MyObj) -> None:
        """Write to artifact store"""
        super().handle_return(my_obj)
        with fileio.open(os.path.join(self.artifact.uri, "data.txt"), "w") as f:
            f.write(my_obj.name)

Let's try running our pipeline again, make sure to specify the return materializer for all steps that return the object_type associated with your custom materializer. 

In [7]:
pipe(
    step1=step1().with_return_materializers(MyMaterializer),
    step2=step2()
).run()

Creating run for pipeline: `pipe`
Cache enabled for pipeline `pipe`
Using stack `secrets_stack2` to run pipeline `pipe`...
Step `step1` has started.
Step `step1` has finished in 0.124s.
Step `step2` has started.
aria_the_cat
Step `step2` has finished in 0.088s.
Pipeline run `pipe-01_Apr_22-15_04_05_078985` has finished in 0.223s.


And voilà, your pipeline is up and running 